## Writing tests for python packages.

Intro on testing suites: \
Neuroscientists aren't good at writing code. Generally, their goals are to collect data, apply some simple analyses to that data, then present the conclusions of that analysis. With time, a neuroscientist's personal bespoke analysis workflow might become crystallized into a pipeline with particular inputs and outputs. This is where the neuroscientist's expertise ends and they need help. Say they want to share their analysis method with others in their lab or in other labs. Now, the data that will be going into the code will derive from different sources, and the outputs must remain accurate. In addition, the code will be used on different operating systems, use different versions of underlying libraries (e.g. different numpy releases), and be subject to various maintenance iterations like bug fixes and new features. How can we ensure that the code always works or at least fails gracefully no matter the inputs, environment, or changes that occur. Software engineers have solved this problem of 'continuous integration': just write tests. Neuroscientists have generally never heard of a testing suite; their code just works on their system with their data. The goal of this coding exercise is to help an imaginary neuroscientist. You will consider the various usecases of a piece of code, understand it's functions and sharp edges, and then implement a simple testing suite that will allow for continugous integration of the code.

Given the various levels of complexity that can be applied in solving this problem, we provide 3 levels that you can choose from. The higher levels require using specific frameworks like pytest, setuptools, and pip, which are not prerequisites and can be learned on the job. If you are someone with experience using these tools, please consider skipping levels 1 and 2, and going straight to 3.

### Level 1:
- Look at the code below in the cell labeled `## codeblock 2`. We will treat this code as if it were written by a neuroscientist who asked you for help in preparing this code to share with others. Read the docstring and look over the code briefly before reading on. Your task is to do the following:
    - **Improve the code**. Consider all the ways that it can fail and do the following:
        - **check inputs**: Modify the function so that it checks it's input variables to ensure that no errors will occur and that the formatting makes sense.
        - **improve numerical handling**: Improve the code to make it more robust to numerical errors (what if the input contains all NaNs, all zeros, has unexpected values, etc.). Take the liberty of assuming how you think each case should be handled; comment your code noting your assumptions.
    - **Write test functions**. Each function should either throw a descriptive error for how the test failed (if it fails), else it should do nothing (if it passes). Follow the style guidelines for writing tests in the `pytest` package (https://docs.pytest.org/en/7.1.x/getting-started.html) and see a good example of a repo that implements tests here: (https://github.com/RichieHakim/ROICaT). Do the following:
        - **ill-conditioned inputs**: Write a function that tests that the `pipeline` function can detect and throw an error upon receiving inputs that are ill-conditioned to run through the function. Again, feel free to make assumptions about what constitutes 'ill-conditioned' input data. One example might be if the `'trial_on'` variable were missing from the CSV file (see the docstring in the function).
        - **accuracy**: Write some code that can generate fake data that will result in known outputs. Then write a test function that generates some fake data with known properties, passes that data through the `pipeline` function, and then checks that the output is accurate.

Other info:
- For level 1, feel free to simply write these functions and demonstrate their utility within this notebook.
- In `## codeblock 1` you'll find some functions that are used to generate the input files to `pipeline`. These are meant to simulate data deriving from the neuroscientist's experiments. That said, it might be useful to draw some inspiration from the `make_fake_voltage_trace` function in deriving a function that can generate data with known properties for accuracy testing.

### Level 2:

Implement the level 1 code, and deploy it within a github repo. The repo should take on the following organization:
- `my_pipeline`
    - `__init__.py`
    - `pipeline.py`
- `tests`
    - `test_all.py`
- `conftest.py`
- `setup.py`
- `setup.cfg`
- `README.md`
- `MANIFEST.in`
- `requirements.txt`

- Rely on the structure found in this repo to guide you: https://github.com/RichieHakim/ROICaT
- Please either make your repo public or if it is private, just add all of our usernames as collaborators: richiehakim, celiaberon, jbwallace123, bernardosabatini
- The repo should be pip installable via `pip install .`
- Calling pytest from CLI should work via `pytest`
- If you get stuck in this exercise, reach out to us and we might be able to help.

### Level 3:

This one is a bit different (and was written by a different person). None of the below code is needed.
The task is to write a testing suite for an existing package: https://github.com/jbwallace123/sabatini-glm-workflow. Fork the repo, determine it's function, and implement a set of tests that can be called by pytest from CLI.

The repo is a simple analysis pipeline made by neuroscientist in our lab. Essentially it implements a generalized linear model on data collected from one of our tasks. The input data are expected to be timeseries with some particular columns. See the code in this folder to infer what these data types and shapes are: https://github.com/jbwallace123/sabatini-glm-workflow/tree/main/sglm.
Data should be in a csv format with at least a SessionName, TrialNumber, Timestamp column. The predictors need to be binary and the response needs to be continuous variable.

The tests should do the following: 
1. Make sure that the input data is the correct type.
2. Make sure that the outputs are correct and do not vary wildly.

In [493]:
## codeblock 1

def make_fake_parameters(filepath_save=r'~/Desktop/parameters.json'):
    """
    Make a fake parameters file.

    Args:
        filepath_save (str):
            Filepath to save the parameters file.
    """
    import json
    import numpy as np

    parameters = {
        'sample_rate': float(1000),
        'threshold': float(np.random.randint(0, 100))
    }

    with open(filepath_save, 'w') as f:
        json.dump(parameters, f)

def make_fake_data_file(filepath_save=r'~/Desktop/data.csv'):
    """
    Make a fake data file.

    Args:
        filepath_save (str):
            Filepath to save the data file.
    """
    import pandas as pd
    import numpy as np

    N = 100000

    data = {}
    
    data['trial_on'] = ((np.arange(N) % 1000) > 500).astype(np.bool_)
    data['reward_on'] = ((np.arange(N) % 1000) > 750).astype(np.bool_)
    data['light_on'] = ((np.arange(N) % 2000) > 900).astype(np.bool_)

    voltages = {f"neuron_{ii + 1}": make_fake_voltage_trace(N) for ii in range(7)}

    ## Make dataframe with named columns
    data.update(voltages)
    data_df = pd.DataFrame(data)

    ## Save to CSV
    data_df.to_csv(filepath_save, index=False)

def make_fake_voltage_trace(n):
    """
    Poisson spike train with some noise.
    Baseline voltage: -70 mV
    Spike voltage: +80 mV
    Spike width: 2 ms
    Noise: Gaussian with mean 0 and std 5 mV
    """
    import numpy as np
    import scipy.signal

    baseline_voltage = -70
    spike_voltage = 100
    threshold_voltage = -45
    refractory_period = 40 # ms
    spike_width = 1 # ms (std)
    noise_mean = 0
    variance = 8
    noise_std = 10

    sample_rate = 10000 # Hz

    ## make a randomly varying trace
    voltage_trace = np.random.normal(0, variance, n) + baseline_voltage
    spike_bool = voltage_trace > threshold_voltage
    ## remove spikes within the refractory period
    spike_times = np.where(spike_bool)[0].astype(np.float32)
    spike_times[np.where(np.diff(spike_times, prepend=0) < refractory_period)[0]] = np.nan
    spike_times = spike_times[~np.isnan(spike_times)]
    ## make a new boolean trace with the refractory spikes removed
    spike_bool = np.isin(np.arange(n), spike_times)
    ## convolve boolean with gaussian to approximate the timecourse of a spike
    spike_kernel = scipy.signal.windows.gaussian(spike_width/1000 * sample_rate * 5, spike_width/1000 * sample_rate) * spike_voltage
    spike_kernel = spike_kernel / np.max(spike_kernel)
    voltage_trace += (np.convolve(spike_bool, spike_kernel, mode='same') * spike_voltage) + np.random.normal(noise_mean, noise_std, n)

    return voltage_trace


In [497]:
make_fake_data_file('/Users/richardhakim/Desktop/data.csv')

make_fake_parameters('/Users/richardhakim/Desktop/params.json')

In [498]:
## codeblock 2

def pipeline(filepath_data, filepath_parameters):
    """
    Fake neuroscience analysis pipeline.
    This code counts the number of spikes fired by neurons.
    The input data are timeseries traces of the voltage of individual neurons.
    The code applies some signal processing techniques to clean the data, 
     then extracts and counts the number of peaks, which are assumed to be spikes.
    The output is a dataframe with the total number of spikes summed across all neurons
     during the trial periods for different experimental conditions.

    Required libraries:
        - pandas
        - numpy
        - scipy
        - matplotlib

    Args:
        filepath_data (str):
            Filepath to CSV file.
            CSV file should have one row of headers.
            Each column should correspond to a timeseries trace and the first row should be the name of that trace.
            The data should contain the 3 critical columns that correspond to temporal epochs within the experiment: 
                - 'trial_on'
                - 'reward_on'
                - 'light_on'
            All other columns are expected to be the voltage traces of individual neurons.
        filepath_parameters (str):
            Filepath to JSON file that can be read into python as a python dictionary object.
            The parameters are expected to contain the following elements with defined 'key' names:
                - 'sample_rate' (float): frequency at which data samples were collected.
                - 'threshold' (float): voltage above which the voltage must reach to be considered a valid spike.
    """
    import pandas as pd
    import json
    import numpy as np

    data = pd.read_csv(filepath_data)
    with open(filepath_parameters, 'r') as f:
        parameters = json.load(f)

    keys_trial = ['trial_on', 'reward_on', 'light_on']
    t, r, l = (data[key] for key in keys_trial)
    keys_neurons = [key for key in data.keys() if key not in keys_trial]
    st = [count_spikes(data[key]) for key in keys_neurons] # 'spike_times'
    st_cat = np.concatenate(st)

    bool_to_idx = lambda x: np.where(x)[0]
    idx_conditions = {
        't': bool_to_idx(t), ## trial_on
        'r': bool_to_idx(r), ## reward_on
        'l': bool_to_idx(l), ## light_on
        'tr': bool_to_idx(t * r),
        'tl': bool_to_idx(t * l),
        'rl': bool_to_idx(r * l),
        'trl': bool_to_idx(t * r * l),
    }    

    ns_conditions = {key: np.isin(st_cat, val).sum() for key, val in idx_conditions.items()}
    return ns_conditions
    

def count_spikes(trace, sample_rate=10000, threshold=10):
    ## smooth the trace
    w = sample_rate / 500 # roughly the n_samples of a spike
    w = int(w + np.remainder(w, 2)) # make odd
    trace_smooth = scipy.signal.savgol_filter(
        x=trace,
        window_length=w, 
        polyorder=2,
    )

    ## find peaks (spike times)
    peaks, _ = scipy.signal.find_peaks(
        x=trace_smooth,
        height=threshold,
        distance=(2/1000 * sample_rate),
    )
    
    return peaks

In [501]:
pipeline(filepath_data='/Users/richardhakim/Desktop/data.csv', filepath_parameters='/Users/richardhakim/Desktop/params.json')

{'t': 306, 'r': 147, 'l': 361, 'tr': 147, 'tl': 191, 'rl': 106, 'trl': 106}